In [1]:
from scipy.integrate import odeint
from scipy.optimize import minimize
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

# Gathering Data

In [2]:
mat = loadmat('20210201_run3_data.mat')
# this gives us a dictionary of arrays.. here are the values that we need to extract to look at
# crop to entries 11600:91200 (275s to 595s) in the Run3 file
# X- OXT_E
# Y- OXT_N
# yaw- OXT_aYaw
# Vx- OXT_vxCG
# Vy- OXT_vyCG
# yawrate- OXT_nYaw
# steer angle- CMD_aWheelFSteer (this is in degrees so * pi/180 to get radians) 
# accel request- CMD_gLongRequest (in g so *9.8 to get m/s)
# real ax- OXT_axCG
# real ay- OXT_ayCG

In [3]:
time = mat['Time'][:,0]
x = mat['OXT_E']
y = mat['OXT_N']
psi = mat['OXT_aYaw']
vx = mat['OXT_vxCG']
vy = mat['OXT_vyCG']
r = mat['OXT_nYaw']
steer = mat['CMD_aWheelFSteer'] # in degrees so * pi/180 to get radians
D = mat['CMD_gLongRequest'] # in g so *9.8 to get m/s
real_ax = mat['OXT_axCG']
real_ay = mat['OXT_ayCG']

In [4]:
atime = 275.036 # start the data
btime = 595

In [5]:
df = pd.DataFrame(x[(x[:,0] >= atime) & (x[:,0] <= btime)], columns=['time', 'x'])
data = [y, psi, vx, vy, r, steer, D, real_ax, real_ay]
labels = ['y', 'psi', 'vx', 'vy', 'r', 'steer', 'D', 'real_ax', 'real_ay']

for i in range(len(data)):
    label = labels[i] # get label
#     print(label)
    time_cut_data = data[i][(data[i][:,0] >= atime) & (data[i][:,0] <= btime)] # get 2d array in time frame
    if label in ('steer', 'D'):
        # we need to downsample
        time_cut_data = time_cut_data[::10]
        
    # manually check that they line up correctly
    # print(time_cut_data[:,0][:5])
    # print(time_cut_data[:,0][-5:])
    # print(time_cut_data.shape)
    
    values = time_cut_data[:,1] # now extract the actual column value
    
    if label == 'steer':
        # multiply pi/180 to get rad
        print(values[:5])
        values *= math.pi / 180
        print(values[:5])
    elif label == 'D':
        # multiple by 9.8 to get m/s^2
        print(values[:5])
        values *= 9.8
        print(values[:5])
    
    df[labels[i]] = time_cut_data[:,1] # now extract the actual column value
    
df.to_csv('20210201_parsed_run3_data.csv')

[0.06146778 0.06052634 0.06451372 0.071417   0.07229017]
[0.00107282 0.00105638 0.00112598 0.00124646 0.0012617 ]
[-0.04278078 -0.04276823 -0.04215042 -0.04583811 -0.04424571]
[-0.4192516  -0.41912864 -0.41307414 -0.44921352 -0.43360798]


In [6]:
df

,time,x,y,psi,vx,vy,r,steer,D,real_ax,real_ay
0,275.036,-101.308861,19.530313,-2.566862,14.049075,0.064813,-0.00306,0.001073,-0.419252,-0.2477,0.5202
1,275.076,-101.002190,19.059580,-2.566963,14.046067,0.070796,-0.00887,0.001056,-0.419129,-0.1003,0.1653
2,275.116,-100.695587,18.588818,-2.567172,14.046994,0.067643,-0.00909,0.001126,-0.413074,-0.0117,0.0054
3,275.156,-100.389305,18.117285,-2.567444,14.055491,0.059404,-0.01306,0.001246,-0.449214,-0.0894,-0.0761
4,275.196,-100.082977,17.645878,-2.567532,14.051016,0.066986,0.00144,0.001262,-0.433608,-0.0839,0.5010
...,...,...,...,...,...,...,...,...,...,...,...
7995,594.836,-167.020935,120.190155,-2.566120,13.627166,0.029047,-0.02381,-0.002339,-4.265322,-5.6695,-0.0485
7996,594.876,-166.724564,119.735992,-2.567150,13.422322,0.042470,-0.02686,-0.001983,-4.274560,-4.9157,0.0203
7997,594.916,-166.433014,119.288536,-2.568222,13.224626,0.034855,-0.02117,-0.001433,-4.278177,-5.2175,-0.7925
7998,594.956,-166.146347,118.847389,-2.569167,13.019039,0.037049,-0.01664,-0.000933,-4.222267,-5.6539,0.3684
